In [49]:
import psycopg2
import random
import time
from datetime import datetime
from pytz import timezone
import uuid # 导入uuid模块

# 数据库连接信息
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
# 设置 search_path 
cur.execute("SET search_path TO raw_cdc;")


In [48]:
# import os
# # 设置 CSV 文件目录
# csv_directory = "csv/"

# def load_csv_to_postgres(csv_file, table_name):
#     with open(csv_file, 'r') as f:
#         cur.copy_expert(f"COPY {table_name} FROM STDIN WITH CSV HEADER DELIMITER ','", f)
#     conn.commit()

# # 遍历目录中的所有 CSV 文件并加载到对应的表中
# for filename in os.listdir(csv_directory):
#     if filename.endswith(".csv"):
#         csv_file = os.path.join(csv_directory, filename)
#         table_name = os.path.splitext(filename)[0]  # 文件名去掉扩展名作为表名
#         print(f"Loading {csv_file} into table {table_name}...")
#         load_csv_to_postgres(csv_file, table_name)
#         print(f"Loaded {csv_file} into table {table_name} successfully!")


Loading csv/customers.csv into table customers...
Loaded csv/customers.csv into table customers successfully!
Loading csv/products.csv into table products...
Loaded csv/products.csv into table products successfully!
Loading csv/merchants.csv into table merchants...
Loaded csv/merchants.csv into table merchants successfully!
Loading csv/transactions.csv into table transactions...
Loaded csv/transactions.csv into table transactions successfully!


In [50]:
# 设置参数
loop_interval = 0  # 循环间隔（秒）
loop_count = 1000  # 循环次数
batch_size = 100  # 每次插入的记录数

In [51]:
def insert_transactions(loop_interval,loop_count,batch_size):
    # 循环 loop_count 次
    for i in range(loop_count):
        try:
            for j in range(1, batch_size + 1):
                # 从现有表中随机选择有效的客户、产品和商家
                cur.execute("SELECT * FROM customers ORDER BY RANDOM() LIMIT 1;")
                v_existing_customer = cur.fetchone()


                cur.execute("SELECT * FROM products ORDER BY RANDOM() LIMIT 1;")
                v_existing_product = cur.fetchone()


                cur.execute("SELECT * FROM merchants ORDER BY RANDOM() LIMIT 1;")
                v_existing_merchant = cur.fetchone()


                # 生成新的交易 ID（唯一）
                v_new_transaction_id = f"TX{int(datetime.now().timestamp())}{j}"
                # 生成新的交易 ID（UUID 格式） 
                v_new_transaction_id = str(uuid.uuid4())

                # 生成纽约时区的当前日期和时间
                nyc_time = datetime.now(timezone('America/New_York'))
                v_transaction_date = nyc_time.date()
                v_transaction_time = nyc_time.strftime('%H:%M:%S')

                # 生成随机数量（1 到 7 之间）
                v_quantity = random.randint(1, 7)

                # 获取产品价格并计算总价格
                v_product_price = v_existing_product[3]  # 价格在第4列
                # if not is_number(v_product_price):
                #     continue  # 跳过非数值记录
                v_total_price = float(v_product_price) * v_quantity

                # 随机选择交易卡类型
                v_transaction_card = random.choice(['American Express', 'Visa', 'Mastercard', 'Discover'])

                # 随机选择交易类别
                v_transaction_category = 'Purchase' if random.random() < 0.8 else 'Refund'

                # 将新交易插入 transactions 表
                cur.execute("""
                    INSERT INTO transactions (
                        transaction_id, customer_id, product_id, merchant_id, transaction_date, transaction_time, quantity, total_price, transaction_card, transaction_category
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
                """, (
                    v_new_transaction_id, v_existing_customer[0], v_existing_product[0],
                    v_existing_merchant[0], v_transaction_date, v_transaction_time,
                    v_quantity, v_total_price, v_transaction_card, v_transaction_category
                ))

            # 每批 batch_size 行后提交
            conn.commit()

            # 插入下一批之前等待 loop_interval 秒
            time.sleep(loop_interval)
        
        except Exception as e:
            print(f"Error occurred: {e}")
            conn.rollback()  # 回滚当前事务

In [54]:
# 调用函数插入交易数据
insert_transactions(loop_interval,loop_count,batch_size)

In [ ]:
# 关闭游标和连接
cur.close()
conn.close()